In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
from pathlib import Path

root = "https://ai.stanford.edu/~amaas/data/sentiment/"
filename = "aclImdb_v1.tar.gz"
filepath = tf.keras.utils.get_file(filename, root + filename, extract=True,
                                   cache_dir=".")
path = Path(filepath).with_name("aclImdb")
path

84125825/84125825 [==============================] - 130s 2us/step


WindowsPath('datasets/aclImdb')